In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import integrate
## Parameters and initial conditions
r= 2.0
h= 4.0
K= 10.0
dil= 10.0
T_rd= 4.0
N_initial= [0.2, 0.2]
## Defining the ordinary differential equation system (ODEs)
def derivative(N,t,r,h,K,a,b):
  N1= N[0]
  N2= N[1]
  dotN1= r*N1*(1-(N1+N2)/K)
  dotN2= (1-a+a/(1+math.exp(-3*(N1-1.5))))*(1-b+b/(1+math.exp(-10*((N1/(N1+N2))-0.5))))*h*r*N2*(1-(N1+N2)/K)
  return np.array([dotN1, dotN2])
## Solving the ODEs
Num_points= 4000
T_final= 200.
t= np.linspace(0.,T_final,Num_points)
itvl= int(T_final/T_rd)
Num_points_itvl= int(Num_points/itvl)
#
txt_file= open("output.txt", "w")
txt_file.write("a\tb\tDFreq\n")
v1_list= np.linspace(0., 1., num= 101)
v2_list= np.linspace(0., 1., num= 101)
#
for k in range(0, v1_list.size):
  for j in range(0, v2_list.size):
    for i in range(int(itvl)):
      t_sub= t[i*Num_points_itvl: (i+1)*Num_points_itvl]
      if i==0:
        solved= integrate.odeint(derivative, N_initial, t_sub, args= (r,h,K,v1_list[k],v2_list[j]))
      else:
        N1, N2= solved.T
        N_last= [N1[-1]/ dil, N2[-1]/ dil]
        solved_temp= integrate.odeint(derivative, N_last, t_sub, args= (r,h,K,v1_list[k],v2_list[j]))
        solved= np.concatenate((solved, solved_temp), axis= 0)
    N1, N2= solved.T
    Freq= N2/(N1+N2)
    Freq_diff= max(Freq[-81:-1])- min(Freq[-81: -1])
    txt_file.write("%f\t%f\t%f\n" % (v1_list[k], v2_list[j], Freq_diff))
#
txt_file.close()
